In [ ]:
from llama_index.graph_stores.neo4j import Neo4jGraphStore
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
NEO4J_URI=''
NEO4J_USERNAME=''
NEO4J_PASSWORD=''
AURA_INSTANCEID=''
AURA_INSTANCENAME=''

graph = Neo4jGraphStore(NEO4J_USERNAME, NEO4J_PASSWORD, NEO4J_URI)

In [ ]:
graph.schema

'Node properties are the following:\nPatient {id: STRING, name: STRING, gender: STRING, age: INTEGER, location: STRING},Prescription {id: STRING, end_date: STRING, instructions: STRING, dosage: STRING, medication: STRING, time: STRING, start_date: STRING, frequency: STRING}\nRelationship properties are the following:\n\nThe relationships are the following:\n(:Patient)-[:HAS_PRESCRIPTION]->(:Prescription),(:Prescription)-[:FOR_PATIENT]->(:Patient)'

In [ ]:
graph.query(
    """
MATCH (p:Patient)
where p.gender='Male'
return p
"""
)


[{'p': {'gender': 'Male',
   'name': 'John Doe',
   'location': 'City A',
   'id': 'P1',
   'age': 30}},
 {'p': {'gender': 'Male',
   'name': 'Michael Johnson',
   'location': 'City C',
   'id': 'P3',
   'age': 55}},
 {'p': {'gender': 'Male',
   'name': 'David Lee',
   'location': 'City B',
   'id': 'P5',
   'age': 35}},
 {'p': {'gender': 'Male',
   'name': 'William Taylor',
   'location': 'City A',
   'id': 'P7',
   'age': 42}},
 {'p': {'gender': 'Male',
   'name': 'Daniel Anderson',
   'location': 'City C',
   'id': 'P9',
   'age': 60}}]

In [ ]:
from langchain.chains import GraphCypherQAChain
from langchain_groq import ChatGroq
from langchain_community.llms import OpenAIChat
from langchain.graphs import Neo4jGraph

In [ ]:
import neo4j
from langchain_groq import ChatGroq
graph=Neo4jGraph(NEO4J_URI,NEO4J_USERNAME,NEO4J_PASSWORD)

In [ ]:
llm = ChatGroq(temperature=0, groq_api_key=groq_api, model_name="mixtral-8x7b-32768")

In [ ]:
graph.schema

'Node properties are the following:\nPatient {id: STRING, name: STRING, gender: STRING, age: INTEGER, location: STRING},Prescription {id: STRING, end_date: STRING, instructions: STRING, dosage: STRING, medication: STRING, time: STRING, start_date: STRING, frequency: STRING}\nRelationship properties are the following:\n\nThe relationships are the following:\n(:Patient)-[:HAS_PRESCRIPTION]->(:Prescription),(:Prescription)-[:FOR_PATIENT]->(:Patient)'

In [ ]:
# p1= "Consider yourself as a CYPHER language expert for Neo4j. Based on the below, generate the query. Do not include any comments or explanations"
# p2 = GraphCypherQAChain.from_llm(
#     ChatGroq(temperature=0, groq_api_key=groq_api, model_name="mixtral-8x7b-32768"), graph=graph, verbose=True)

p2 = GraphCypherQAChain.from_llm(
    OpenAIChat(temperature=0,), graph=graph, verbose=True)

langchain.chains.graph_qa.cypher.GraphCypherQAChain

In [ ]:
chain.apply()
# chain.invoke("What are the medicines and the frequency for Jane Smith?")




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Patient)-[:HAS_PRESCRIPTION]->(pres:Prescription)-[:FOR_PATIENT]->(n)
WHERE p.name = 'Jane Smith' AND n.name = 'Jane Smith'
RETURN DISTINCT pres.medication, pres.frequency;
```


DatabaseError: {code: Neo.DatabaseError.Statement.ExecutionFailed} {message: Failed to parse query `MATCH (p:Patient)-[:HAS_PRESCRIPTION]->(pres:Prescription)-[:FOR_PATIENT]->(n)
WHERE p.name = 'Jane Smith' AND n.name = 'Jane Smith'
RETURN DISTINCT pres.medication, pres.frequency;
````.}

In [ ]:
ans['result']

"I'm not able to provide an answer to that question as there is no information provided about Jane Smith's medication periodicity. Without additional information, I am unable to provide a helpful answer.\n\nHowever, I can suggest that you consult with a medical professional or a reliable healthcare source to obtain the most accurate and up-to-date information about medication periodicity for Jane Smith or any other individual. It's important to note that medication regimens can vary greatly depending on individual circumstances, and it's crucial to follow the advice of a qualified healthcare professional to ensure proper care and treatment."

# **Start Here**

In [ ]:
# %pip install groq
# %pip install neo4j
# %pip install langchain
# !pip install openai

In [ ]:
from groq import Groq
import neo4j
from langchain.graphs import Neo4jGraph

In [ ]:
def call_llm(query,question=None):
  groq_api=''

  client = Groq(
      api_key=groq_api,
  )
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": query,
          }
      ],
      model="mixtral-8x7b-32768",
      temperature=0
  )

  return(chat_completion.choices[0].message.content)
  # from openai import OpenAI
  # client = OpenAI(api_key='')
  # response = client.chat.completions.create(
  #     model='gpt-3.5-turbo',
  #     messages=[
  #          {"role": "user", "content": query},
  #         ],
  #     temperature=0)
  # return response.choices[0].message.content

In [ ]:
def load_graph(username,password,uri):
  # graph=(NEO4J_URI,NEO4J_USERNAME,NEO4J_PASSWORD)
  graph= Neo4jGraph(uri,username,password)
  return graph

In [ ]:
def return_query(graph):
  # graph=graph.refresh_schema()
  query="Consider yourself as a Neo4J expert. Given the following schema of a Neo4j graph database, "+ str(graph.get_schema)+" \nWrite a cypher query to answer the following question: \n. "
  question=input()
  query=query+question
  query=query+" Just give the query so that I can copy paste the query and directly execute it. Stricitly no comments or special characters or explantion of any type"
  response=call_llm(query)
  response=response.replace("`",'')
  return question,graph.query(response)


In [ ]:
def first_ans(question, response):
  print(response)
  query=f'''
  Consider you are answering a patient over a phone call
  Following is the Question asked by Patient: {question}
  Following is the answer for the question: {response}.

  Use this information to make remind the person in the question and answer any queries they may have to take his medication. mention the time as well.
  Example reminder: Hi XYZ. This is to remind you that you have to take your abc medication at x o clock. Have you taken it?
  I only want the response. Stricitly no comments or explantion of any type
  '''
  ans=call_llm(query)
  return ans

In [ ]:
def conversation(bot):
  query='Consider you are a medicine assistant reminder and call agent serving medical queries over phone. Complete the follwing conversation playing the role of medicine reminder assistant. If user has any queries, inform them that you will pass it on to the doctor.\n\n I only want the response. Stricitly no comments, no notes or explantion of any type'
  user_msg= input()
  while True:
    if user_msg:
      user_msg='Patient: '+user_msg
      bot_msg= 'Medicine assistant reminder: '+ bot
      # print(user_msg)
      query= query+'\n'+ bot_msg+'\n'+user_msg
      print(query)
      bot=call_llm(query)
      print("Bot: "+bot)
      user_msg= input()
    else:
      break


In [ ]:
NEO4J_URI=''
NEO4J_USERNAME=''
NEO4J_PASSWORD=''
AURA_INSTANCEID=''
AURA_INSTANCENAME=''

graph=load_graph(NEO4J_USERNAME,NEO4J_PASSWORD,NEO4J_URI)
# print(graph.get_structured_schema)
response=return_query(graph)
bot1_ans=(first_ans(response[0],response[1]))
print(bot1_ans)
conversation(bot1_ans)

Jane Smith
[{'prescription': {'end_date': '2024-04-15', 'instructions': 'Take on an empty stomach', 'dosage': '20mg', 'medication': 'MedB', 'id': 'Rx2', 'time': '06:00 PM', 'start_date': '2024-03-15', 'frequency': 'Twice daily'}}]
Hi Jane, this is a reminder to take MedB. The dosage is 20mg, to be taken twice daily on an empty stomach. The first dose should have been taken on March 15, 2024, and the last dose will be on April 15, 2024. The next dose is scheduled for now, at 06:00 PM. Have you taken it?

